This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [3]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case

#API_KEY = 'your-key-here'

# Import QUANDL_API_KEY from the .env file
from dotenv import load_dotenv
load_dotenv(verbose=True)

import os
API_KEY = os.getenv("QUANDL_API_KEY")

print(type(API_KEY))
#print(API_KEY)


<class 'str'>


Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [29]:
# First, import the relevant modules
import requests
from datetime import datetime, timedelta


In [30]:
# Make a sample API Call for 1 day worth of data

#get yesterday's date with datetime and timedelta modules
yesterday = datetime.now() - timedelta(days=1)

In [31]:
# set API call parameters
database_code = 'FSE' # Frankfurt Stock Exchange Database
dataset_code = 'AFX_X' #Ticker symbol for Carl Zeiss Meditec
start_date = yesterday.strftime('%Y-%m-%d') #convert to string and format to yyyy-mm-dd

url = 'https://www.quandl.com/api/v3/datasets/'+database_code+'/'+dataset_code+'/data.json'

payload = {'api_key': API_KEY, 'start_date': start_date}

r = requests.get(url, params=payload)

In [64]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r_dict = r.json()
#print(r_dict)
#print(r_dict['dataset_data'])
print(r_dict['dataset_data']['column_names'])
#print(r_dict['dataset_data']['data'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [57]:
#Collect Data for 2017
# set API call parameters
database_code = 'FSE' # Frankfurt Stock Exchange Database
dataset_code = 'AFX_X' #Ticker symbol for Carl Zeiss Meditec
start_date = '2017-01-01'

url = 'https://www.quandl.com/api/v3/datasets/'+database_code+'/'+dataset_code+'/data.json'

payload = {'api_key': API_KEY, 'start_date': start_date}

r = requests.get(url, params=payload)

In [58]:
# save the respnse data to a dict
d = r.json()

In [66]:
lst = d['dataset_data']['data']
#print(lst)

In [81]:
#highest_opening = max(l[1] for l in lst)
#lowest_opening = min(l[1] for l in lst)

In [106]:
opening = []
closing = []
high = []
low = []

for l in lst :
    #print(l[1])
    opening.append(l[1])  
    high.append(l[1])
    low.append(l[1])
    closing.append(l[4])


In [102]:
print(len(opening))
print(len(closing))
print(len(high))
print(len(low))

845
845
845
845


In [103]:
clean_opening = list(filter(lambda x: x is not None, opening))
clean_closing = list(filter(lambda x: x is not None, closing))
clean_high = list(filter(lambda x: x is not None, high))
clean_low = list(filter(lambda x: x is not None, low))

In [104]:
type(list(clean_low))

list

In [105]:
print(len(clean_opening))
print(len(clean_closing))
print(len(clean_high))
print(len(clean_low))

522
845
522
522


In [ ]:
# We have a problem 
# it seems that the closing entry does not have any empty values 
# I need to figure out how to handle the None value in this dataset

In [84]:
highest_opening = max(filter(lambda x: x is not None, opening))
lowest_opening = min(filter(lambda x: x is not None, opening))
largest_daily_change = max([high[i] - low[i] for i in range(len(high))])

TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'